In [ ]:

from tqdm import tqdm
import pickle
import pandas as pd
from pathlib import Path
import numpy as np
import copy
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

sys.path.append('/mnt/c/Users/lslat/Code/')
sys.path.append('/mnt/c/Users/lslat/QiskitProjects/VariationalWavefunction/')

from quantum_kernel.code.utils import get_dataset
from quantum_kernel.code.visualization_utils import aggregate_folder
from quantum_kernel.code.kernel_measures import kernel_geometry
from quantum_kernel.code.utils import compute_additional_fields

font = {'family': 'DejaVu Sans',
        'weight': 'bold',
        'size': 18}

rc('font', **font)

tqdm.pandas()

In [ ]:
# Get kernel results for the different ml models and dataset.d
dfs = {}
dataset_name='fashion-mnist'
folders = [
    ("/mnt/c/Users/lslat/Data/QK_project/results/Sparse_IQPm/"+dataset_name,'Sparse_IQPm'),
    ("/mnt/c/Users/lslat/Data/QK_project/results/rbf/"+dataset_name, "rbf"),
    ("/mnt/c/Users/lslat/Data/QK_project/results/laplacian/"+dataset_name, 'laplacian'),
]

for folder, kernel_name in folders:
    if dataset_name in dfs:
        temp = aggregate_folder(folder, dataset_name, kernel_name=kernel_name)[
            dataset_name]
        dfs[dataset_name] = pd.concat([dfs[dataset_name], temp])
    else:
        dfs[dataset_name] = aggregate_folder(
            folder, dataset_name, kernel_name=kernel_name)[dataset_name]

save_folder='/mnt/c/Users/lslat/Data/QK_project/figs/'


In [ ]:
df = dfs[dataset_name]
sparse_iqpm_df = df[(df['kernel_name'] == 'Sparse_IQPm') & (df['dataset_dim'] == 10)]
